In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle

# ===============================
# 1️⃣ Text Cleaning Function
# ===============================
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()  # lowercase + trim
    text = re.sub(r"<.*?>", " ", text)  # remove HTML
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)  # remove URLs
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text)  # keep letters, numbers, + - x % . /
    text = re.sub(r"\s+", " ", text).strip()  # collapse spaces/newlines
    return text

# ===============================
# 2️⃣ Load Tokenizer and Model
# ===============================
TOKENIZER_PATH = "tokenizer.pkl"   # Path where tokenizer was saved
LSTM_MODEL_PATH = "lstm_model.h5"  # Path where LSTM model was saved

# Load tokenizer
with open(TOKENIZER_PATH, 'rb') as f:
    tokenizer = pickle.load(f)

# Load LSTM model
model = load_model(LSTM_MODEL_PATH)

# Define max sequence length used during training
MAX_SEQ_LEN = 100  # same as training

# ===============================
# 3️⃣ Predictor Function
# ===============================
def predictor(sample_id, catalog_content, image_link=None):
    # Clean the text
    text = clean_text(catalog_content)

    # Convert to sequence
    seq = tokenizer.texts_to_sequences([text])

    # Pad sequence
    seq_pad = pad_sequences(seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

    # Predict price
    price_pred = model.predict(seq_pad).flatten()[0]

    # Ensure positive price
    return float(max(price_pred, 0))

# ===============================
# 4️⃣ Load Test Data
# ===============================
DATASET_FOLDER = 'dataset/'
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

# Fill missing catalog_content if any
test['catalog_content'] = test['catalog_content'].fillna("")

# ===============================
# 5️⃣ Apply Predictor to Test Set
# ===============================
test['price'] = test.apply(
    lambda row: predictor(row['sample_id'], row['catalog_content'], row['image_link']),
    axis=1
)

# ===============================
# 6️⃣ Prepare Submission
# ===============================
submission = test[['sample_id', 'price']]
output_filename = os.path.join(DATASET_FOLDER, 'test_out.csv')
submission.to_csv(output_filename, index=False)

print(f"Predictions saved to {output_filename}")
print(f"Total predictions: {len(submission)}")
print(f"Sample predictions:\n{submission.head()}")


In [ ]:
import re
import string
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
import pandas as pd

# Show full content in each cell
pd.set_option('display.max_colwidth', None)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/train.csv')
train.head()

sample_id  \
0      33127   
1     198967   
2     261251   
3      55858   
4     292686   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   catalog_content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Item Name: Salerno Cookies, The Original Butter Cookies, 8 Ounce (Pack of 4)\nBullet Point 1: Original Butter Cookies: Classic butter cookies made with real butter\nBullet Point 2: Variety Pack: Includes 4 boxes with 32 cookies total\nBullet Point 3: Occasion Perfect: Delicious cookies for birthdays, weddings, anniversaries\nBullet Point 4: Shareable Treats: Fun to give and enjoy with friends and family\nBullet Point 5: Sal

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/test.csv')
test.head()

sample_id  \
0     100179   
1     245611   
2     146263   
3      95658   
4      36806   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      catalog_content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Item Name: Rani 14-Spice Eshamaya's Mango Chutney (Indian Preserve) 10.5oz (300g) Glass Jar, Ready to eat, Vegan ~ Gluten Free, All Natural, NON-GMO\nBullet Point 1: You'll LOVE our 14-Spice Eshamaya's Mango Chutney Chutney by Rani Brand--Here's Why:\nBullet Point 2: 🥭All Natural, Non-GMO, No Preservatives, Gluten Free PREMIUM Gourmet Food Grade Chutney.\nBullet Point 3: 🥭Authentic Family Rani Recipe. Rani is a USA based company selling Indian Foods for over 40 years, buy with confidence!\nBullet Point 4: 🥭Great Spicy Taste, Awesome with Wine & Cheese! Packaged in a re-closeable glass jar\nBullet Point 5: 🥭Net Wt. 10.5oz - Authentic Product of India\nProduct Description: Mango chutney is made from diced green mangoes cooked with sugar. Spices are added to distinguish the different types of chutneys. Hot mango chutney is spiced with chilli powder which give the chutney heat and a deep amber color. Sweet Mango chutney allows the pallet to appreciate the flavor of the mango, adding a hint of vinegar for sourness, while major grey chutney mixes raisins, ginger powder, chilli powder and an array of spices to give complexity to the chutney. Try Rani Chutney's with pizza, salads, burgers, sandwiches, cheeses and anything else you can imagine!\nValue: 10.5\nUnit: Ounce\n   
1  Item Name: Natural MILK TEA Flavoring extract by HALO PANTRY (2oz bottle) | Perfect for customizing ANY baking, candy, dessert or drink recipes | Dairy Free, Dye-Free, Gluten Free, Zero Sugars, Zero Calories, Gourmet (Pack of 1)\nBullet Point 1: Authentic Tasting, Asian-Inspired Natural flavorings made exclusively for use in any baking, cooking, dessert, candy, or drink recipes.\nBullet Point 2: NATURAL : Gluten-Free, Dairy-F

In [ ]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()

Train shape: (75000, 4)
Test shape: (75000, 3)


sample_id  \
0      33127   
1     198967   
2     261251   
3      55858   
4     292686   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   catalog_content  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Item Name: La Victoria Green Taco Sauce Mild, 12 Ounce (Pack of 6)\nValue: 72.0\nUnit: Fl Oz\n   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Item Name: Salerno Cookies, The Original Butter Cookies, 8 Ounce (Pack of 4)\nBullet Point 1: Original Butter Cookies: Classic butter cookies made with real butter\nBullet Point 2: Variety Pack: Includes 4 boxes with 32 cookies total\nBullet Point 3: Occasion Perfect: Delicious cookies for birthdays, weddings, anniversaries\nBullet Point 4: Shareable Treats: Fun to give and enjoy with friends and family\nBullet Point 5: Sal

In [ ]:
train.isnull().sum()

sample_id          0
catalog_content    0
image_link         0
price              0
dtype: int64

In [ ]:
# Function to show details
def dataset_info(name, df):
    print(f"📊 Dataset: {name}")
    print("="*50)
    print("Shape (rows, columns):", df.shape)
    print("\nColumn Names:", df.columns.tolist())
    print("\nData Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nFirst 5 Rows:\n", df.head())
    print("\n" + "="*50 + "\n")

In [ ]:
# Show details for both
dataset_info("Trainung Data Information", train)


📊 Dataset: Trainung Data Information
Shape (rows, columns): (75000, 4)

Column Names: ['sample_id', 'catalog_content', 'image_link', 'price']

Data Types:
 sample_id            int64
catalog_content     object
image_link          object
price              float64
dtype: object

Missing Values:
 sample_id          0
catalog_content    0
image_link         0
price              0
dtype: int64

First 5 Rows:
    sample_id  \
0      33127   
1     198967   
2     261251   
3      55858   
4     292686   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
dataset_info("Testing Data Information", test)

📊 Dataset: Testing Data Information
Shape (rows, columns): (75000, 3)

Column Names: ['sample_id', 'catalog_content', 'image_link']

Data Types:
 sample_id           int64
catalog_content    object
image_link         object
dtype: object

Missing Values:
 sample_id          0
catalog_content    0
image_link         0
dtype: int64

First 5 Rows:
    sample_id  \
0     100179   
1     245611   
2     146263   
3      95658   
4      36806   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
# ===============================
# 1️⃣ Text Cleaning Function
# ===============================
def clean_text(text):

    if not isinstance(text, str):
        return ""

    # Lowercase and trim
    text = text.lower().strip()

    # Remove HTML tags
    text = re.sub(r"<.*?>", " ", text)

    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)

    # Remove emojis and non-standard unicode symbols
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002700-\U000027BF"  # dingbats
        u"\U0001F900-\U0001F9FF"  # supplemental symbols
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Keep letters, numbers, and meaningful symbols: + - x % . /
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text)

    # Collapse multiple spaces/newlines into single space
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [ ]:
# ===============================
# 2️⃣ Apply Preprocessing
# ===============================
train['clean_text'] = train['catalog_content'].fillna("").apply(clean_text)
test['clean_text'] = test['catalog_content'].fillna("").apply(clean_text)

In [ ]:
# ===============================
# 3️⃣ Tokenization & Padding
# ===============================
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 100  # max words per product description

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train['clean_text'])

X_train_seq = tokenizer.texts_to_sequences(train['clean_text'])
X_test_seq = tokenizer.texts_to_sequences(test['clean_text'])

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

In [ ]:
# ===============================
# 4️⃣ Train/Validation Split
# ===============================
y_train = train['price'].values
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_pad, y_train, test_size=0.2, random_state=42
)

In [ ]:
# ===============================
# 5️⃣ Build LSTM Model
# ===============================
EMBEDDING_DIM = 100

model = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LEN),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  # regression output
])

model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ===============================
# 6️⃣ Train the Model
# ===============================
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=128
)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 190s 396ms/step - loss: 17.2182 - val_loss: 16.2276
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 195s 382ms/step - loss: 15.3250 - val_loss: 14.2869
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 386ms/step - loss: 13.5109 - val_loss: 13.3824
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 179s 381ms/step - loss: 11.9495 - val_loss: 12.4446
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 180s 385ms/step - loss: 10.6854 - val_loss: 12.0903


In [ ]:

# ===============================
# 7️⃣ Validate
# ===============================
y_val_pred = model.predict(X_val).flatten()
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {mae:.2f}")

# ===============================
# 8️⃣ Predict on Test Set
# ===============================
test_preds = model.predict(X_test_pad).flatten()
test_preds = np.maximum(test_preds, 0)  # Ensure positive prices

469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step
Validation MAE: 12.09
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 115s 49ms/step


In [ ]:
# Performance Metrics

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) / 2
    smape_val = np.mean(numerator / denominator) * 100
    return smape_val

# Example
y_true = np.array([100, 200, 300])
y_pred = np.array([120, 190, 310])

print("SMAPE:", smape(y_true, y_pred))

SMAPE: 8.862903944871157


In [ ]:

# ===============================
# 10 Prepare Submission
# ===============================
submission = pd.DataFrame({
    "sample_id": test['sample_id'],
    "price": test_preds
})

submission.to_csv("test_out.csv", index=False)
print("Submission file 'test_out.csv' created successfully!")

Submission file 'test_out.csv' created successfully!


In [ ]:
# Imports
import os
import re
import string
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
import tensorflow as tf

In [ ]:
# Load data
train = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Amazon_ML/student_resource/dataset/test.csv')

In [ ]:
# Clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    emoji_pattern = re.compile("[" u"\U0001F600-\U0001F64F" u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF" u"\U0001F1E0-\U0001F1FF"
                               u"\U00002700-\U000027BF" u"\U0001F900-\U0001F9FF" "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

train['clean_text'] = train['catalog_content'].fillna("").apply(clean_text)
test['clean_text'] = test['catalog_content'].fillna("").apply(clean_text)

In [ ]:
# Tokenization
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train['clean_text'])

X_train_seq = tokenizer.texts_to_sequences(train['clean_text'])
X_test_seq = tokenizer.texts_to_sequences(test['clean_text'])

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

y_train = train['price'].values
X_tr, X_val, y_tr, y_val = train_test_split(X_train_pad, y_train, test_size=0.2, random_state=42)

In [ ]:
# Build GRU Model
EMBEDDING_DIM = 100

model = Sequential([
    Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LEN),
    GRU(128, return_sequences=True),
    Dropout(0.3),
    GRU(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train
history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=7,
    batch_size=128
)

Epoch 1/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 352s 735ms/step - loss: 17.1934 - val_loss: 15.4151
Epoch 2/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 367s 702ms/step - loss: 13.9003 - val_loss: 13.2270
Epoch 3/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 306s 653ms/step - loss: 11.9492 - val_loss: 12.5705
Epoch 4/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 297s 633ms/step - loss: 10.7748 - val_loss: 11.9134
Epoch 5/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 301s 642ms/step - loss: 10.2126 - val_loss: 11.7708
Epoch 6/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 299s 638ms/step - loss: 9.5700 - val_loss: 11.7950
Epoch 7/7
469/469 ━━━━━━━━━━━━━━━━━━━━ 390s 783ms/step - loss: 9.3734 - val_loss: 11.7277


In [ ]:
# Evaluate
y_val_pred = model.predict(X_val).flatten()
mae = mean_absolute_error(y_val, y_val_pred)
print(f"Validation MAE: {mae:.2f}")

469/469 ━━━━━━━━━━━━━━━━━━━━ 28s 60ms/step
Validation MAE: 11.73


In [ ]:
# Predict on test set
test_preds = model.predict(X_test_pad).flatten()
test_preds = np.maximum(test_preds, 0)

2344/2344 ━━━━━━━━━━━━━━━━━━━━ 110s 47ms/step


In [ ]:
# Save model and tokenizer
model.save('/content/drive/MyDrive/Amazon_ML/gru_model.h5')

import pickle
with open('/content/drive/MyDrive/Amazon_ML/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# ===========================
# 🔧 CONFIG
# ===========================
DATASET_FOLDER = '/content/drive/MyDrive/Amazon_ML/student_resource/dataset/'
MODEL_PATH = '/content/drive/MyDrive/Amazon_ML/gru_model.h5'
TOKENIZER_PATH = '/content/drive/MyDrive/Amazon_ML/tokenizer.pkl'
MAX_SEQ_LEN = 100  # Make sure this matches your training setting

# ===========================
# 🧹 Clean Function
# ===========================
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower().strip()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    text = re.sub(r"[^a-z0-9\s\+\-x%./]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===========================
# 🚀 Fast Batch Prediction
# ===========================
def run_batch_prediction():
    print("🔄 Loading model and tokenizer...")
    model = load_model(MODEL_PATH)

    with open(TOKENIZER_PATH, 'rb') as f:
        tokenizer = pickle.load(f)

    print("📄 Reading test.csv...")
    test_df = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))

    print("🧹 Cleaning text data...")
    test_df['cleaned_text'] = test_df['catalog_content'].fillna("").apply(clean_text)

    print("🔢 Tokenizing...")
    sequences = tokenizer.texts_to_sequences(test_df['cleaned_text'])
    padded = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

    print("🧠 Predicting...")
    preds = model.predict(padded).flatten()
    test_df['price'] = np.maximum(preds, 0).round(2)

    output_df = test_df[['sample_id', 'price']]
    output_path = os.path.join(DATASET_FOLDER, 'test_out.csv')
    output_df.to_csv(output_path, index=False)

    print(f"✅ Predictions saved to {output_path}")
    print("📊 Sample predictions:\n", output_df.head())

# ===========================
# ✅ Main
# ===========================
if __name__ == "__main__":
    run_batch_prediction()

🔄 Loading model and tokenizer...


📄 Reading test.csv...
🧹 Cleaning text data...
🔢 Tokenizing...
🧠 Predicting...
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 112s 47ms/step
✅ Predictions saved to /content/drive/MyDrive/Amazon_ML/student_resource/dataset/test_out.csv
📊 Sample predictions:
    sample_id  price
0     100179  13.23
1     245611  13.13
2     146263  25.35
3      95658   3.58
4      36806  31.66


In [ ]:
from sklearn.metrics import mean_absolute_error

def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_pred) + np.abs(y_true)) / 2
    return np.mean(numerator / denominator) * 100

In [ ]:
# Predict on validation set
y_val_pred = model.predict(X_val).flatten()

# Calculate metrics
mae = mean_absolute_error(y_val, y_val_pred)
smape_val = smape(y_val, y_val_pred)

print(f"Validation MAE: {mae:.2f}")
print(f"Validation SMAPE: {smape_val:.2f}%")

469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step
Validation MAE: 11.73
Validation SMAPE: 51.74%
